In [19]:
import os
import random
import pygame
import mediapipe as mp
import cv2
import numpy as np
from Fruit import Fruit
from HandTrackingModule import HandDetector  # Assuming you have HandTrackingModule in the same directory
import pymunk
import time

# Pygame 초기화
pygame.init()
pygame.mixer.init()  # 소리 초기화를 위한 라인 추가
pygame.event.clear()

# 창/화면 생성
width, height = 1300, 700
window = pygame.display.set_mode((1200, 700))
pygame.display.set_caption("Fruit Slicer")

# FPS용 시계 초기화
fps = 23
clock = pygame.time.Clock()

# 이미지 리사이징
imgGameOver = pygame.transform.scale(pygame.image.load(r"C:\Users\bluecom011\Desktop\Fruit/fru.jpg").convert(), (1300, 800))

# Mediapipe 포즈 클래스 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# 웹캠 설정
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)  # 너비
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)  # 높이

# Hand detector 초기화
detector = HandDetector(maxHands=1, detectionCon=0.8)

# 물리 공간 초기화
space = pymunk.Space()
space.gravity = 0.0, -1000.0

# 게임 변수 및 파라미터
timeTotal = 10 # 게임 총 시간(초)
fruitList = [] # 생성된 과일 객체를 저장할 리스트
timeGenerator = time.time() # 과일 생성 시간을 추적하기 위한 변수
timeStart = time.time() # 게임 시작 시간을 추적하기 위한 변수
gameOver = False # 게임 종료 여부를 나타내는 변수
totalScore = 0 #총 점수를 누적할 변수
blue = (255, 127, 0) # 파이게임에서 사용할 파란색

# 과일 경로 목록
pathFruitFolder = r"C:\Users\bluecom011\Desktop\Fruit/Fruits"
pathListFruit = os.listdir(pathFruitFolder)

# 전체 점수를 누적할 변수
totalScore = 0

# 게임 초기화 함수
def reset_game():
    global totalScore, timeGenerator, timeStart, currentRound, gameOver, fruitList
    totalScore = 0
    timeGenerator = time.time()
    timeStart = time.time()
    currentRound = 1
    gameOver = False
    fruitList = []

# 과일 생성 함수
def generate_fruit():
    # 랜덤한 크기 및 과일 이미지 경로 선택
    randomScale = round(random.uniform(0.6, 0.8), 2)
    randomFruitPath = pathListFruit[random.randint(0, len(pathListFruit) - 1)]
    # 폭탄인 경우와 그렇지 않은 경우에 따라 소리 경로 선택
    pathSoundSlice = r'C:\Users\bluecom011\Desktop\Fruit/explosion.wav' if "bomb" in randomFruitPath else r'C:\Users\bluecom011\Desktop\Fruit/slice.wav'
    # Fruit 클래스의 객체를 생성하여 fruitList에 추가
    fruitList.append(Fruit(space, path=os.path.join(pathFruitFolder, randomFruitPath),
                           grid=(4, 4), animationFrames=14, scale=randomScale,
                           pathSoundSlice=pathSoundSlice))

# 리스타트 버튼 함수
def restart_button():
    global gameOver, totalScore, timeGenerator, timeStart, fruitList, currentRound
    # 버튼 크기 및 위치 설정
    button_width, button_height = 200, 50
    button_x, button_y = (width - button_width) // 2, height // 2 + 100
    # 버튼을 그립니다.
    pygame.draw.rect(window, (0, 255, 0), (button_x, button_y, button_width, button_height))
    # 마우스의 위치 및 클릭 여부 확인
    mouse_x, mouse_y = pygame.mouse.get_pos()
    click, _, _ = pygame.mouse.get_pressed()
    # 마우스가 버튼 위에 있는지 확인하고 클릭하면 게임을 다시 시작합니다.
    if button_x < mouse_x < button_x + button_width and button_y < mouse_y < button_y + button_height:
        pygame.draw.rect(window, (0, 200, 0), (button_x, button_y, button_width, button_height))
        if click:
            # 게임 변수 초기화
            gameOver = False
            totalScore = 0
            timeGenerator = time.time()
            timeStart = time.time()
            pygame.mixer.music.load(r"C:\Users\bluecom011\Desktop\Fruit/background.mp3")
            pygame.mixer.music.play(-1)
            currentRound = 1
            fruitList = []
    # 버튼에 텍스트 추가
    font = pygame.font.Font(None, 36)
    restart_text = font.render("Restart", True, (255, 255, 255))
    text_rect = restart_text.get_rect(center=(button_x + button_width // 2, button_y + button_height // 2))
    window.blit(restart_text, text_rect)

# 게임 루프 함수
def game_loop():
    global gameOver, totalScore, timeGenerator, timeStart, currentRound
    # 현재 라운드 및 총 라운드 설정
    currentRound = 1
    totalRounds = 100
    # 배경 음악 로드 및 재생
    pygame.mixer.music.load(r"C:\Users\bluecom011\Desktop\Fruit/background.mp3")
    pygame.mixer.music.play(-1)

    while cap.isOpened() and currentRound <= totalRounds:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return
        # 웹캠에서 프레임 읽기
        if gameOver is False:
            success, img = cap.read()
            img = cv2.flip(img, 1)
            h, w = img.shape[:2]
            # 이미지 컬러 변환
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            # Mediapipe를 사용하여 포즈 검출
            keypoints = pose.process(img)
             # 이미지를 원래대로 변환
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            # 포즈 랜드마크 가져오기
            lm = keypoints.pose_landmarks
            lmPose = mp_pose.PoseLandmark

            try:
                # 오른손 검지 손가락 끝의 x, y 좌표 계산
                index_finger_tip_x = int(lm.landmark[lmPose.RIGHT_INDEX].x * w)
                index_finger_tip_y = int(lm.landmark[lmPose.RIGHT_INDEX].y * h)
                # 손 검출
                hands, img = detector.findHands(img)

                if hands:
                    # 손이 검출되면 손가락 끝의 좌표 업데이트
                    lmlist = hands[0]['lmList']
                    ind_x, ind_y = lmlist[8][0], lmlist[8][1]
                    index_finger_tip_x, index_finger_tip_y = ind_x, ind_y
                # 손가락 끝에 파란색 동그라미 그리기
                cv2.circle(img, (index_finger_tip_x, index_finger_tip_y), 20, blue, -1)
            except AttributeError:
                # 손가락이 감지되지 않으면 계속 진행
                continue
            # OpenCV 이미지를 Pygame Surface로 변환
            imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            imgRGB = np.rot90(imgRGB)
            frame = pygame.surfarray.make_surface(imgRGB).convert()
            # 이미지 좌우 반전(거울모드)
            frame = pygame.transform.flip(frame, True, False)
            # 프레임을 창에 표시
            window.blit(frame, (0, 0))
            # 과일 생성 및 손가락 위치 업데이트
            if time.time() - timeGenerator > 1:
                generate_fruit()
                timeGenerator = time.time()

            x, y = index_finger_tip_x, index_finger_tip_y
            # 생성된 과일들에 대해 처리
            for i, fruit in enumerate(fruitList):
                if fruit:
                    fruit.draw(window)
                    checkSlice = fruit.checkSlice(x, y)
                    # 과일이 폭탄일 경우 게임 종료
                    if checkSlice == 2:
                        gameOver = True
                        pygame.mixer.music.stop()
                    # 과일이 슬라이스되었을 경우
                    if checkSlice == 1:
                        fruitList[i] = False
                        totalScore += 1
            # 남은 시간 계산
            timeLeft = int(timeTotal - (time.time() - timeStart))

            # 시간이 다 된 경우 다음 라운드 시작 또는 게임 종료
            if timeLeft <= 0:
                if currentRound < totalRounds:
                    currentRound += 1
                    timeStart = time.time()
                    pygame.mixer.music.load(r"C:\Users\bluecom011\Desktop\Fruit/background.mp3")
                    pygame.mixer.music.play(-1)

                    # 다음 라운드 시작 메시지 출력
                    font = pygame.font.Font(None, 60)
                    round_start_text = font.render(f"{currentRound}Round Start!", True, (255,0, 0))
                    text_rect = round_start_text.get_rect(center=(width // 2, height // 2))
                    window.blit(round_start_text, text_rect)
                    pygame.display.update()
                    pygame.time.delay(500)

                else:
                    # 게임 종료
                    gameOver = True
                    pygame.mixer.music.stop()

            # 점수 및 남은 시간 표시
            font = pygame.font.Font(None, 60)
            textScore = font.render(str(totalScore), True, blue)
            textTime = font.render(str(timeLeft), True, blue)
            window.blit(textScore, (225, 35))
            window.blit(textTime, (1000, 37))

        else:
            # 게임 오버 화면 표시
            window.blit(imgGameOver, (0, 0))

            # "You Lose!" 텍스트 및 점수, 라운드 표시
            font = pygame.font.Font(None, 150)
            textLose = font.render("You Lose!", True, (0, 0, 0))
            textYour = font.render("Your Score:", True, (0, 0, 0))
            textyou = font.render("Your Round:", True, (0, 0, 0))
            textScore = font.render(str(totalScore), True, (0, 0, 0))
            textRound = font.render(str(currentRound), True, (0, 0, 0))
            window.blit(textLose, (400, 43))
            window.blit(textYour, (350, 143))
            window.blit(textScore, (950, 143))
            window.blit(textRound, (1040, 243))
            window.blit(textyou, (350, 243))
            # 리스타트 버튼 표시
            restart_button()

        pygame.display.update()
        clock.tick(fps)
        space.step(1 / fps)
    # Pygame 종료
    pygame.quit()

if __name__ == "__main__":
    game_loop()


In [3]:
import os
import random
import pygame
import mediapipe as mp
import cv2
import numpy as np
from Fruit import Fruit
from HandTrackingModule import HandDetector  # Assuming you have HandTrackingModule in the same directory
import pymunk
import time

# Pygame 초기화
pygame.init()
pygame.mixer.init()  # 소리 초기화를 위한 라인 추가
pygame.event.clear()

# 창/화면 생성
width, height = 1300, 700
window = pygame.display.set_mode((1200, 700))
pygame.display.set_caption("Fruit Slicer")

# FPS용 시계 초기화
fps = 23
clock = pygame.time.Clock()

# 이미지 리사이징
imgGameOver = pygame.transform.scale(pygame.image.load(r"C:\Users\bluecom011\Desktop\Fruit/fru.jpg").convert(), (1300, 800))

# Mediapipe 포즈 클래스 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# 웹캠 설정
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)  # 너비
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)  # 높이

# Hand detector 초기화
detector = HandDetector(maxHands=1, detectionCon=0.8)

# 물리 공간 초기화
space = pymunk.Space()
space.gravity = 0.0, -1000.0

# 게임 변수 및 파라미터
timeTotal = 10  # 게임 총 시간(초)
comboTimeLimit = 1  # Combo 시간 제한(초)
fruitList = []  # 생성된 과일 객체를 저장할 리스트
timeGenerator = time.time()
timeStart = time.time()
gameOver = False
totalScore = 0
blue = (255, 127, 0)
comboCount = 0
lastSliceTime = time.time()

# 과일 경로 목록
pathFruitFolder = r"C:\Users\bluecom011\Desktop\Fruit/Fruits"
pathListFruit = os.listdir(pathFruitFolder)

# 전체 점수를 누적할 변수
totalScore = 0

# 게임 초기화 함수
def reset_game():
    global totalScore, timeGenerator, timeStart, currentRound, gameOver, fruitList, comboCount
    totalScore = 0
    timeGenerator = time.time()
    timeStart = time.time()
    currentRound = 1
    gameOver = False
    fruitList = []
    comboCount = 0  # 콤보 초기화 추가

# 과일 생성 함수
def generate_fruit():
    # 랜덤한 크기 및 과일 이미지 경로 선택
    randomScale = round(random.uniform(0.6, 0.8), 2)
    randomFruitPath = pathListFruit[random.randint(0, len(pathListFruit) - 1)]
    # 폭탄인 경우와 그렇지 않은 경우에 따라 소리 경로 선택
    pathSoundSlice = r'C:\Users\bluecom011\Desktop\Fruit/explosion.wav' if "bomb" in randomFruitPath else r'C:\Users\bluecom011\Desktop\Fruit/slice.wav'
    # Fruit 클래스의 객체를 생성하여 fruitList에 추가
    fruitList.append(Fruit(space, path=os.path.join(pathFruitFolder, randomFruitPath),
                           grid=(4, 4), animationFrames=14, scale=randomScale,
                           pathSoundSlice=pathSoundSlice))

# 리스타트 버튼 함수
# 리스타트 버튼 함수
def restart_button():
    global gameOver, totalScore, timeGenerator, timeStart, fruitList, currentRound, comboCount
    # 버튼 크기 및 위치 설정
    button_width, button_height = 200, 50
    button_x, button_y = (width - button_width) // 2, height // 2 + 100
    # 버튼을 그립니다.
    pygame.draw.rect(window, (0, 255, 0), (button_x, button_y, button_width, button_height))
    # 마우스의 위치 및 클릭 여부 확인
    mouse_x, mouse_y = pygame.mouse.get_pos()
    click, _, _ = pygame.mouse.get_pressed()
    # 마우스가 버튼 위에 있는지 확인하고 클릭하면 게임을 다시 시작합니다.
    if button_x < mouse_x < button_x + button_width and button_y < mouse_y < button_y + button_height:
        pygame.draw.rect(window, (0, 200, 0), (button_x, button_y, button_width, button_height))
        if click:
            # 게임 변수 초기화
            reset_game()
            # 나머지 초기화 로직 추가
            # 게임 변수 초기화
            gameOver = False
            totalScore = 0
            timeGenerator = time.time()
            timeStart = time.time()
            pygame.mixer.music.load(r"C:\Users\bluecom011\Desktop\Fruit/background.mp3")
            pygame.mixer.music.play(-1)
            currentRound = 1
            fruitList = []
    # 버튼에 텍스트 추가
    font = pygame.font.Font(None, 36)
    restart_text = font.render("Restart", True, (255, 255, 255))
    text_rect = restart_text.get_rect(center=(button_x + button_width // 2, button_y + button_height // 2))
    window.blit(restart_text, text_rect)

# 게임 루프 함수
def game_loop():
    global gameOver, totalScore, timeGenerator, timeStart, currentRound, comboCount, lastSliceTime
    # 현재 라운드 및 총 라운드 설정
    currentRound = 1
    totalRounds = 100
    comboCount = 0 
    # 배경 음악 로드 및 재생
    pygame.mixer.music.load(r"C:\Users\bluecom011\Desktop\Fruit/background.mp3")
    pygame.mixer.music.play(-1)

    while cap.isOpened() and currentRound <= totalRounds:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return
        # 웹캠에서 프레임 읽기
        if gameOver is False:
            success, img = cap.read()
            img = cv2.flip(img, 1)
            h, w = img.shape[:2]
            # 이미지 컬러 변환
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            # Mediapipe를 사용하여 포즈 검출
            keypoints = pose.process(img)
             # 이미지를 원래대로 변환
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            # 포즈 랜드마크 가져오기
            lm = keypoints.pose_landmarks
            lmPose = mp_pose.PoseLandmark

            try:
                # 오른손 검지 손가락 끝의 x, y 좌표 계산
                index_finger_tip_x = int(lm.landmark[lmPose.RIGHT_INDEX].x * w)
                index_finger_tip_y = int(lm.landmark[lmPose.RIGHT_INDEX].y * h)
                # 손 검출
                hands, img = detector.findHands(img)

                if hands:
                    # 손이 검출되면 손가락 끝의 좌표 업데이트
                    lmlist = hands[0]['lmList']
                    ind_x, ind_y = lmlist[8][0], lmlist[8][1]
                    index_finger_tip_x, index_finger_tip_y = ind_x, ind_y
                # 손가락 끝에 파란색 동그라미 그리기
                cv2.circle(img, (index_finger_tip_x, index_finger_tip_y), 20, blue, -1)
            except AttributeError:
                # 손가락이 감지되지 않으면 계속 진행
                continue
            # OpenCV 이미지를 Pygame Surface로 변환
            imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            imgRGB = np.rot90(imgRGB)
            frame = pygame.surfarray.make_surface(imgRGB).convert()
            # 이미지 좌우 반전(거울모드)
            frame = pygame.transform.flip(frame, True, False)
            # 프레임을 창에 표시
            window.blit(frame, (0, 0))
            # 과일 생성 및 손가락 위치 업데이트
            if time.time() - timeGenerator > 1:
                generate_fruit()
                timeGenerator = time.time()

            x, y = index_finger_tip_x, index_finger_tip_y
            # 생성된 과일들에 대해 처리
            comboText = None
            for i, fruit in enumerate(fruitList):
                if fruit:
                    fruit.draw(window)
                    checkSlice = fruit.checkSlice(x, y)

                    if checkSlice == 2:
                        gameOver = True
                        pygame.mixer.music.stop()

                    if checkSlice == 1:
                        fruitList[i] = False
                            # Update the scoring logic
                        if comboCount > 0:
                            totalScore += 2  # Increase the score by 2 during a combo
                        else:
                            totalScore += 1

                        # 현재 시간 기록
                        currentTime = time.time()

                        # 일정 시간 안에 슬라이스가 발생하면 Combo!
                        if currentTime - lastSliceTime <= comboTimeLimit:
                            comboCount += 1
                            comboText = f"Combo x{comboCount}"

                        # 마지막 슬라이스 시간 갱신
                        lastSliceTime = currentTime
            # 남은 시간 계산
            timeLeft = int(timeTotal - (time.time() - timeStart))

            # 시간이 다 된 경우 다음 라운드 시작 또는 게임 종료
            if timeLeft <= 0:
                if currentRound < totalRounds:
                    currentRound += 1
                    timeStart = time.time()
                    pygame.mixer.music.load(r"C:\Users\bluecom011\Desktop\Fruit/background.mp3")
                    pygame.mixer.music.play(-1)

                    # 다음 라운드 시작 메시지 출력
                    font = pygame.font.Font(None, 60)
                    round_start_text = font.render(f"{currentRound}Round Start!", True, (255,0, 0))
                    text_rect = round_start_text.get_rect(center=(width // 2, height // 2))
                    window.blit(round_start_text, text_rect)
                    pygame.display.update()
                    pygame.time.delay(500)

                else:
                    # 게임 종료
                    gameOver = True
                    pygame.mixer.music.stop()
            if comboText:
                font = pygame.font.Font(None, 60)
                textCombo = font.render(comboText, True, (255, 255, 0))
                window.blit(textCombo, (width // 2 - 80, 100))
            # 점수 및 남은 시간 표시
            font = pygame.font.Font(None, 60)
            textScore = font.render(str(totalScore), True, blue)
            textTime = font.render(str(timeLeft), True, blue)
            window.blit(textScore, (225, 35))
            window.blit(textTime, (1000, 37))

        else:
            # 게임 오버 화면 표시
            window.blit(imgGameOver, (0, 0))

            # "You Lose!" 텍스트 및 점수, 라운드 표시
            font = pygame.font.Font(None, 150)
            textLose = font.render("You Lose!", True, (0, 0, 0))
            textYour = font.render("Your Score:", True, (0, 0, 0))
            textyou = font.render("Your Round:", True, (0, 0, 0))
            textScore = font.render(str(totalScore), True, (0, 0, 0))
            textRound = font.render(str(currentRound), True, (0, 0, 0))
            window.blit(textLose, (400, 43))
            window.blit(textYour, (350, 143))
            window.blit(textScore, (950, 143))
            window.blit(textRound, (1040, 243))
            window.blit(textyou, (350, 243))
            # 리스타트 버튼 표시
            restart_button()

        pygame.display.update()
        clock.tick(fps)
        space.step(1 / fps)
    # Pygame 종료
    pygame.quit()

if __name__ == "__main__":
    game_loop()
